In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""\
This module contains classes defining various types of options.
These classes inherits methods from the parent class `Instrument`,
providing a consistent framework for option instruments.
"""

__author__      = None
__copyright__   = None


import sys
sys.path.insert(0,'../src')

from engines.blackscholes import BlackScholesEngine
from instruments.option import EuropeanVanillaOption
from instruments.option import EuropeanVanillaOptions
from instruments.base import Instrument
from engines.heston import HestonEngine
from processes.heston import HestonProcess
from engines.base import Engine
import numpy as np
import typing


***Generating Heston-Model***

In [ ]:
s0 = 100.
risk_free_rate = 0.025
v0 = 0.0175 # initial variance
sigma = 0.5751 # vol of vol
kappa  =  1.5768   # rate of mean reversion
theta = 0.0398 # mean reversion, long run variance
rho = -0.5711 # correlation between the stock price and its variance
phi = 0.05**2 # deterministic displacement
std_asy = np.sqrt(theta * sigma**2 / (2 * kappa))  # asymptotic standard deviation

# as shown in https://arxiv.org/vc/arxiv/papers/1502/1502.02963v1.pdf (pag. 27)
mu = 0.08 # real world drift rate
lambda_h = 0.25 # market price of volatility risk
kappa_rw = kappa - lambda_h
theta_rw = theta * (kappa_rw + lambda_h) / kappa_rw

heston_engine = HestonEngine \
(
    theta = theta, kappa = kappa, sigma = sigma, rho = rho,
    v0 = v0, phi= phi, risk_free_rate = risk_free_rate, s0 = s0
)


***Generating Paths from Heston MonteCarlo***

In [ ]:
%%time
st_h, sigma_h, rho_h = HestonProcess.paths(heston_engine, n_paths=10000, n_steps=251, horizon=15, measure="rw")


CPU times: total: 250 ms
Wall time: 269 ms


In [ ]:
eu_p_pool = EuropeanVanillaOptions \
([
    EuropeanVanillaOption(s0=s0, strike=k, tau=0.3, flag=-1) \
        for s0, k in zip(np.arange(50,180,0.01), np.arange(20,220,0.01))
])


In [ ]:
%%time
eu_p_pool.set_npv(heston_engine)

CPU times: total: 1.86 s
Wall time: 1.88 s


In [ ]:
eu_p_pool.options

[<Instrument.EuropeanVanillaOption(s0=50.0, strike=20.0, tau=0.3, flag='p', sigma=0.0, quote=6.963247756175406e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.01, strike=20.01, tau=0.3, flag='p', sigma=0.0, quote=6.993410295308422e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.019999999999996, strike=20.020000000000003, tau=0.3, flag='p', sigma=0.0, quote=7.023146508799982e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.029999999999994, strike=20.030000000000005, tau=0.3, flag='p', sigma=0.0, quote=7.053344575069787e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.03999999999999, strike=20.040000000000006, tau=0.3, flag='p', sigma=0.0, quote=7.084217656938563e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.04999999999999, strike=20.050000000000008, tau=0.3, flag='p', sigma=0.0, quote=7.115517064448795e-10)>,
 <Instrument.EuropeanVanillaOption(s0=50.05999999999999, strike=20.06000000000001, tau=0.3, flag='p', sigma=0.0, quote=7.148130976020184e-10)>,
 <Instrument.EuropeanVanillaOption(s